<a href="https://colab.research.google.com/github/AKAGUZZ/Tomato-Detection-with-YOLOv8/blob/main/k_fold_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas ultralytics pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.5/877.5 kB 22.4 MB/s eta 0:00:00


In [ ]:
import os

# Lista de rutas a las carpetas de cada clase
class_folders = [
    r'/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro',
    r'/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_semimaduro',
    r'/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_verde',
    r'/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/mixed'
]

# Extensiones de imágenes permitidas
image_extensions = {".jpg", ".jpeg", ".png"}

# Función para verificar que cada imagen tenga su etiqueta correspondiente
def verify_images_and_labels(class_folder):
    image_folder = os.path.join(class_folder, 'images')
    label_folder = os.path.join(class_folder, 'labels')

    # Listas para almacenar imágenes y etiquetas
    image_files = []
    label_files = []

    # Recorre los archivos en la carpeta de imágenes
    for filename in os.listdir(image_folder):
        name, ext = os.path.splitext(filename)
        if ext.lower() in image_extensions:
            image_files.append(name)

    # Recorre los archivos en la carpeta de etiquetas
    for filename in os.listdir(label_folder):
        name, ext = os.path.splitext(filename)
        if ext.lower() == ".txt":
            label_files.append(name)

    # Comparación de archivos
    missing_labels = set(image_files) - set(label_files)  # Imágenes sin etiquetas
    missing_images = set(label_files) - set(image_files)  # Etiquetas sin imágenes

    # Resultados
    print(f"\nEn la carpeta: {class_folder}")
    print(f"Total de imágenes: {len(image_files)}")
    print(f"Total de etiquetas: {len(label_files)}")

    if missing_labels:
        print(f"Imágenes sin etiquetas correspondientes: {missing_labels}")
    else:
        print("Todas las imágenes tienen su etiqueta.")

    if missing_images:
        print(f"Etiquetas sin imágenes correspondientes: {missing_images}")
    else:
        print("Todas las etiquetas tienen su imagen.")


# Verifica todas las carpetas de clases
for class_folder in class_folders:
    verify_images_and_labels(class_folder)



En la carpeta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro
Total de imágenes: 163
Total de etiquetas: 163
Todas las imágenes tienen su etiqueta.
Todas las etiquetas tienen su imagen.

En la carpeta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_semimaduro
Total de imágenes: 110
Total de etiquetas: 110
Todas las imágenes tienen su etiqueta.
Todas las etiquetas tienen su imagen.

En la carpeta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_verde
Total de imágenes: 121
Total de etiquetas: 121
Todas las imágenes tienen su etiqueta.
Todas las etiquetas tienen su imagen.

En la carpeta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/mixed
Total de imágenes: 124
Total de etiquetas: 124
Todas las imágenes tienen su etiqueta.
Todas las etiquetas tienen su imagen.


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Fri Oct 25 01:11:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from pathlib import Path

dataset_path = Path("/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset")
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'

In [ ]:
import yaml

yaml_file = "/content/drive/MyDrive/ITSZ/k_fold/Project/dataset.yaml"  # your data YAML with data directories and names dictionary
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes.keys())

Inicializar un DataFrame vacío

In [ ]:
import pandas as pd
indx = [label.stem for label in labels]  # IDs de archivo sin extensión
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

Contar las instancias de cada clase en los archivos de etiquetas

In [ ]:
from collections import Counter

for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ")[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

<ipython-input-8-6f088342b013>:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`


**K-FOLD CROSS VALIDATION**

Calcular la distribución de clases por fold / Crear los splits del K-Fold

In [ ]:
#K-FOLD
#Calcular la distribución de clases por fold / Crear los splits del K-Fold
from sklearn.model_selection import KFold

ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

Crear el DataFrame que asigna cada imagen a su fold

In [ ]:
# Crear el DataFrame que asigna cada imagen a su fold
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    # Asignación corregida usando df.loc[]
    folds_df.loc[labels_df.iloc[train].index, f"split_{idx}"] = "train"
    folds_df.loc[labels_df.iloc[val].index, f"split_{idx}"] = "val"

Calcular la distribución de las etiquetas en cada fold

In [ ]:
# Calcular la distribución de las etiquetas en cada fold
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

Crear directorios y archivos YAML

In [ ]:
import datetime

supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted(dataset_path.rglob(f"*{ext}")))

# Verificar si se han encontrado imágenes
if not images:
    print("No se encontraron imágenes con las extensiones soportadas.")
else:
    print(f"Se encontraron {len(images)} imágenes.")

# Create the necessary directories and dataset YAML files (modified)
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories (no separate 'images' or 'labels' folders)
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train").mkdir(parents=True, exist_ok=True)
    (split_dir / "val").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

Se encontraron 518 imágenes.


Copiar imágenes y etiquetas a sus respectivas carpetas

In [ ]:
# Copiar imágenes y etiquetas a sus respectivas carpetas
import shutil

# Verificar que imagen y etiqueta coinciden antes de copiar
for image in images:
    # Buscar la etiqueta correspondiente a la imagen
    corresponding_label = next(dataset_path.rglob(f"labels/{image.stem}.txt"), None)

    if corresponding_label is None:
        print(f"Etiqueta faltante para la imagen {image.name}, omitiendo...")
    else:
        print(f"Etiqueta encontrada para {image.name}")
        # Aquí puedes copiar la imagen y la etiqueta a la carpeta correspondiente

    for split, k_split in folds_df.loc[image.stem].items():
        to_path = save_path / split / k_split  # Directorio destino (train o val)

        # Verifica que la imagen y la etiqueta existan y estén emparejadas
        if image.exists() and corresponding_label.exists():
            print(f"Copiando imagen: {image} y etiqueta: {corresponding_label} a {to_path}")
            shutil.copy(image, to_path / image.name)
            shutil.copy(corresponding_label, to_path / corresponding_label.name)
        else:
            print(f"Faltan pares de archivos para {image.stem}, omitiendo...")

print("Proceso de copia completado.")

Etiqueta encontrada para jitomate_maduro01.jpg
Copiando imagen: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro/images/jitomate_maduro01.jpg y etiqueta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro/labels/jitomate_maduro01.txt a /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_1/train
Copiando imagen: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro/images/jitomate_maduro01.jpg y etiqueta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro/labels/jitomate_maduro01.txt a /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_2/val
Copiando imagen: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro/images/jitomate_maduro01.jpg y etiqueta: /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/jitomate_maduro/labels/jitomate_maduro01.txt a /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_3/train
Copiand

Verificar que las carpetas de validación en los splits estén correctas

In [ ]:
import os
from pathlib import Path
import shutil

# Función para verificar que cada imagen tenga su etiqueta correspondiente
def verify_image_label_pairs(split_folder):
    missing_pairs = []
    for split in split_folder.glob('*'):
        for folder_type in ['train', 'val']:  # Revisar carpetas de train y val
            images_path = split / folder_type
            for image_file in images_path.glob("*.jpg"):  # o cualquier otra extensión de imagen
                label_file = images_path / f"{image_file.stem}.txt"
                if not label_file.exists():
                    print(f"Etiqueta faltante para la imagen {image_file.name}, omitiendo...")
                    missing_pairs.append(image_file.name)
                if not image_file.exists():
                    print(f"Imagen faltante para la etiqueta {label_file.name}, omitiendo...")
                    missing_pairs.append(label_file.name)

    if missing_pairs:
        print("\nArchivos con problemas de correspondencia:")
        for missing in missing_pairs:
            print(missing)
    else:
        print("\nTodos los archivos de imágenes y etiquetas tienen sus pares correspondientes.")

# Función para verificar que no haya duplicados en las carpetas de validación de cada split
def verify_unique_validation_files(splits_folder):
    val_files_set = set()
    duplicates = []
    for split in splits_folder.glob('*'):
        val_folder = split / 'val'
        for file in val_folder.glob("*.jpg"):  # Verificamos las imágenes, lo mismo se puede hacer con las etiquetas
            if file.name in val_files_set:
                duplicates.append(file.name)
            else:
                val_files_set.add(file.name)

    if duplicates:
        print("\nArchivos duplicados en las carpetas de validación:")
        for dup in duplicates:
            print(dup)
    else:
        print("\nNo se encontraron archivos duplicados en las carpetas de validación.")

# Ruta donde están los splits (ya generados previamente)
splits_path = Path(r"/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val")

# Verificar imágenes y etiquetas
verify_image_label_pairs(splits_path)

# Verificar duplicados en validación
verify_unique_validation_files(splits_path)


Todos los archivos de imágenes y etiquetas tienen sus pares correspondientes.

No se encontraron archivos duplicados en las carpetas de validación.


In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo de entrenamiento: {device}")

Dispositivo de entrenamiento: cuda


In [ ]:
# Importar la librería ultralytics
from ultralytics import YOLO
import os

# Desactivar Weights & Biases (W&B)
os.environ["WANDB_MODE"] = "disabled"

# Lista de archivos YAML para cada fold
yaml_files = ds_yamls
print(yaml_files)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[PosixPath('/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_1/split_1_dataset.yaml'), PosixPath('/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_2/split_2_dataset.yaml'), PosixPath('/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_3/split_3_dataset.yaml'), PosixPath('/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_4/split_4_dataset.yaml'), PosixPath('/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_5/split_5_dataset.yaml')]


In [ ]:
from ultralytics import YOLO
import os

# Descargar el modelo YOLOv8m
model = YOLO('yolov8m.pt')  # Esto descargará el modelo yolov8m.pt automáticamente

# Verificar que se haya descargado
print(os.path.exists('yolov8m.pt'))  # Debería imprimir True si la descarga fue exitosa

# Iterar sobre los 5 folds y entrenar un modelo por cada split
for i, yaml_file in enumerate(ds_yamls, 1):
    model.train(
        data=yaml_file.as_posix(),  # Ruta del archivo YAML
        epochs=90,                  # Número de épocas
        imgsz=640,                  # Tamaño de imagen
        batch=8,                    # Tamaño del batch
        project=f"fold_{i}_run",    # Carpeta donde se guardarán los resultados del fold actual
        name=f"siu_{i}_run",        # Nombre del experimento
        verbose=True,
        amp=False                   # Desactiva la precisión mixta si no tienes una GPU compatible
    )


100%|██████████| 49.7M/49.7M [00:00<00:00, 229MB/s]


True
Ultralytics 8.3.22 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=90, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=fold_1_run, name=siu_1_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

100%|██████████| 755k/755k [00:00<00:00, 18.8MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

train: Scanning /content/drive/MyDrive/ITSZ/k_fold/Project/Dataset/2024-10-25_5-Fold_Cross-val/split_1/train... 414 images, 0 backgrounds, 0 corrupt: 100%|██████████| 414/414 [00:07<00:00, 52.81it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      14/90      7.15G     0.4491     0.4428     0.9513         53        640: 100%|██████████| 52/52 [00:37<00:00,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.38it/s]

                   all        103        470      0.939      0.887      0.954      0.898



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/90      7.23G     0.4404     0.4801     0.9423         79        640: 100%|██████████| 52/52 [00:37<00:00,  1.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  43%|████▎     | 3/7 [00:01<00:01,  2.28it/s]